<a href="https://colab.research.google.com/github/SAlaMusa/breastcancerdetection/blob/main/ml/aipractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Define the number of samples
n_samples = 1000

# Create a DataFrame with columns
data = pd.DataFrame({
    'Area': np.random.uniform(500, 5000, n_samples),  # Area in square feet
    'Bedrooms': np.random.randint(1, 6, n_samples),  # Number of bedrooms
    'Bathrooms': np.random.randint(1, 5, n_samples),  # Number of bathrooms
    'Age': np.random.randint(1, 50, n_samples),  # Age of the property in years
    'Location': np.random.choice(['Kicukiro', 'Gasabo', 'Nyarugenge', 'Kicukiro'], n_samples)  # Location in Kigali
})

# Generate the target variable (Price) based on other features
data['Price'] = (data['Area'] * np.random.uniform(100, 300, n_samples)) + \
                (data['Bedrooms'] * np.random.uniform(50000, 100000, n_samples)) + \
                (data['Bathrooms'] * np.random.uniform(30000, 80000, n_samples)) - \
                (data['Age'] * np.random.uniform(5000, 20000, n_samples)) + \
                np.random.normal(0, 50000, n_samples)  # Add some noise

# Save the dataset to a CSV file
data.to_csv('kigali_homes.csv', index=False)

In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Load the dataset
data = pd.read_csv('kigali_homes.csv')

# One-hot encode the 'Location' column
encoder = LabelEncoder()
data['Location'] = encoder.fit_transform(data['Location'])
ohe = OneHotEncoder(sparse=False)
location_ohe = ohe.fit_transform(data[['Location']])
data = data.drop('Location', axis=1)
data = pd.concat([data, pd.DataFrame(location_ohe, columns=encoder.classes_)], axis=1)

# Split the data into features (X) and target (y)
X = data.drop('Price', axis=1)
y = data['Price']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=3, learning_rate=0.1)

# Train the model
xgb_model.fit(X_train, y_train)

# Evaluate the model
y_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Mean Squared Error: 65606639388.75736
R-squared: 0.6312259505172093


In [4]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import streamlit as st

# Create a synthetic dataset
n_samples = 1000
data = pd.DataFrame({
    'Area': np.random.uniform(500, 5000, n_samples),
    'Bedrooms': np.random.randint(1, 6, n_samples),
    'Bathrooms': np.random.randint(1, 5, n_samples),
    'Age': np.random.randint(1, 50, n_samples),
    'Location': np.random.choice(['Kicukiro', 'Gasabo', 'Nyarugenge', 'Kicukiro'], n_samples)
})

data['Price'] = (data['Area'] * np.random.uniform(100, 300, n_samples)) + \
                (data['Bedrooms'] * np.random.uniform(50000, 100000, n_samples)) + \
                (data['Bathrooms'] * np.random.uniform(30000, 80000, n_samples)) - \
                (data['Age'] * np.random.uniform(5000, 20000, n_samples)) + \
                np.random.normal(0, 50000, n_samples)

# Preprocess the data
encoder = LabelEncoder()
data['Location'] = encoder.fit_transform(data['Location'])
ohe = OneHotEncoder(sparse=False)
location_ohe = ohe.fit_transform(data[['Location']])
data = data.drop('Location', axis=1)
data = pd.concat([data, pd.DataFrame(location_ohe, columns=encoder.classes_)], axis=1)

# Split the data into features (X) and target (y)
X = data.drop('Price', axis=1)
y = data['Price']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the hyperparameter search space
param_dist = {
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_lambda': [0.1, 1, 10, 100]
}

# Create the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Perform randomized search
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=100, cv=5, scoring='r2', n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Print the best hyperparameters
print(f'Best hyperparameters: {random_search.best_params_}')

# Evaluate the tuned model
xgb_tuned = random_search.best_estimator_
y_pred = xgb_tuned.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Save the model
xgb_tuned.save_model('kigali_homes_model.json')

# Function to make predictions
def predict_price(area, bedrooms, bathrooms, age, location):
    # One-hot encode the location
    location_encoded = encoder.transform([location])[0]
    location_ohe = ohe.transform([[location_encoded]])
    location_ohe = pd.DataFrame(location_ohe, columns=encoder.classes_)

    # Create a DataFrame with the input features
    data = pd.DataFrame([[area, bedrooms, bathrooms, age] + list(location_ohe.iloc[0])],
                         columns=['Area', 'Bedrooms', 'Bathrooms', 'Age'] + list(encoder.classes_))

    # Make the prediction
    prediction = xgb_tuned.predict(data)[0]

    return prediction

# Streamlit app
st.title('Simple Property Valuation')

area = st.number_input('Area (square feet)', min_value=500, max_value=5000, step=100)
bedrooms = st.number_input('Number of Bedrooms', min_value=1, max_value=5, step=1)
bathrooms = st.number_input('Number of Bathrooms', min_value=1, max_value=4, step=1)
age = st.number_input('Age of Property (years)', min_value=1, max_value=50, step=1)
location = st.selectbox('Location', ['Kicukiro', 'Gasabo', 'Nyarugenge', 'Kicukiro'])

if st.button('Predict Price'):
    predicted_price = predict_price(area, bedrooms, bathrooms, age, location)
    st.write(f'Predicted Price: {predicted_price:.2f} Rwandan Francs')

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
2024-05-19 19:13:45.024 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-05-19 19:13:45.026 Session state does not function when running a script without `streamlit run`


Best hyperparameters: {'reg_lambda': 10, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0}
Mean Squared Error: 76252884576.15233
R-squared: 0.5608404671702852
